## scrappingComentYoutube

In [151]:
!pip install selenium --quiet


[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [152]:
import time
from selenium.webdriver import Chrome
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

## Scraping

In [153]:
data=[]

with Chrome(executable_path=r'C:\Program Files\chromedriver.exe') as driver:
    wait = WebDriverWait(driver,10)
    driver.get("https://www.youtube.com/watch?v=lH7J8HuWcQo")

    for item in range(10): 
        wait.until(EC.visibility_of_element_located((By.TAG_NAME, "body"))).send_keys(Keys.END)
        time.sleep(5)

    for comment in wait.until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, "#content"))):
        data.append(comment.text)

C:\Users\asus\AppData\Local\Temp\ipykernel_14096\2771055209.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  with Chrome(executable_path=r'C:\Program Files\chromedriver.exe') as driver:


In [154]:
import pandas as pd   
df = pd.DataFrame(data, columns=['comment'])
df

,comment
0,"ID\nAbaikan navigasi\nLogin\nPemilu 2024, Nama..."
1,
2,Saya heran sebenarnya siapa yg mau calonkan Ga...
3,Tetap amanah dalam mengawal Bangsa sebagai sum...
4,Semoga negara kita dpt pemimipin dan wakilnya ...
...,...
364,
365,
366,
367,


## Preporcessing and Cleaning

Preprocessing

In [155]:
import re, string
import numpy as np

# Text Cleaning
def cleaning(text):
    # HTML Tag Removal

    text = re.compile('<.*?>|&([a-z0-9]+|#[0-9]{1,6}|#x[0-9a-f]{1,6});').sub('', str(text))

    # Case folding
    text = text.lower()

    # Trim text
    text = text.strip()

    # Remove punctuations, karakter spesial, and spasi ganda
    text = re.compile('<.*?>').sub('', text)
    text = re.compile('[%s]' % re.escape(string.punctuation)).sub(' ', text)
    text = re.sub('\s+', ' ', text)

    # Number removal
    text = re.sub(r'\[[0-9]*\]', ' ', text)
    text = re.sub(r'[^\w\s]', '', str(text).lower().strip())
    text = re.sub(r'\d', ' ', text)
    text = re.sub(r'\s+', ' ', text)

    # Mengubah text 'nan' dengan whitespace agar nantinya dapat dihapus
    text = re.sub('nan', '', text)

    return text

In [156]:
!pip install IndoNLP


[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [157]:
from nltk.stem import PorterStemmer
from indoNLP.preprocessing import pipeline, replace_word_elongation, replace_slang, emoji_to_words, remove_html
pipe = pipeline([replace_word_elongation, replace_slang,
                emoji_to_words, remove_html])
# steamming
ps = PorterStemmer()

In [158]:
df['comment (clean)'] = df['comment'].apply(lambda x: pipe(x))
df['comment (clean)'] = df['comment (clean)'].apply(lambda x: cleaning(x))
# ubah empty string menjadi NaN
df['comment (clean)'] = df['comment (clean)'].replace('', np.nan)

In [159]:
print(df.isna().sum())

comment            0
comment (clean)    8
dtype: int64


In [160]:
df.dropna(inplace=True)

In [161]:
df

,comment,comment (clean)
0,"ID\nAbaikan navigasi\nLogin\nPemilu 2024, Nama...",id abaikan navigasi login pemilu nama anies ga...
2,Saya heran sebenarnya siapa yg mau calonkan Ga...,saya heran sebenarnya siapa yang mau calonkan ...
3,Tetap amanah dalam mengawal Bangsa sebagai sum...,tetap amanah dalam mengawal bangsa sebagai sum...
4,Semoga negara kita dpt pemimipin dan wakilnya ...,semoga negara kita dapat pemimipin dan wakilny...
5,Insya Allah penyambung lidah rakyat ketahanan ...,insya allah penyambung lidah rakyat ketaha pan...
...,...,...
357,Ganjar -prabowo tiada lawan istebes,ganjar prabowo tiada lawan istebes
358,Gmn ceritanya?pak ganjar belum di usung sm PDI...,bagaimana ceritanya pak ganjar belum di usung ...
359,Tiga tiganya ga ada yg tak pilih.. berarti ga ...,tiga tiganya enggak ada yang tak pilih berarti...
360,Ganjar,ganjar


In [162]:
# to csv
df.to_csv('datacomment1.csv', index=False)